In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.optim.lr_scheduler import OneCycleLR

# Set up the ResNeXt50-32x4d model as the backbone
model = models.resnext50_32x4d(pretrained=True)
num_classes = 2  # Assuming binary classification, adjust accordingly

# Replace the last fully connected layer to match the number of classes
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Set up the optimizer with SGD and Nesterov momentum
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, nesterov=True)

# Set up the learning rate scheduler using the one-cycle policy
scheduler = OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(train_loader), epochs=8)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the class weights based on the weighting scheme from Cui et al.
class_weights = torch.tensor([1.0, 10.0])  # Adjust the weights based on your specific scenario

# Training loop
for epoch in range(8):
    model.train()
    
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)
        
        # Calculate the loss with class weights
        loss = criterion(outputs, labels)
        weighted_loss = torch.mean(loss * class_weights[labels])
        
        # Backward pass and optimization
        optimizer.zero_grad()
        weighted_loss.backward()
        optimizer.step()
        
        # Update learning rate
        scheduler.step()
